In [1]:
%load_ext autoreload
%autoreload 2
import random
import pprint 
from Mylib import *

Start loading Mylib...
Mylib imported!


## Data Preparation

#### 20대 보건복지부

raw data

In [63]:
# 1. Data Preparation
raw_data = pd.read_pickle('./result/HWC_noun_token_m.pkl')
raw_data = raw_data.sort_values(['CONFER_NUM', 'SEQ_NO'])
raw_data = raw_data.loc[[idx for idx in raw_data.index if raw_data.PARTY.loc[idx] != '非의원']]

load final lda model

In [ ]:
manual_rms = pd.read_csv('./data/190915_용어_수작업.csv') # Manual term removals
manual_rms = manual_rms[manual_rms['code'] == 1]['단어'].values 
manual_rms = [rm for rm in manual_rms if len(rm) == 1]

speakers = set([s for tkns in [spker.split(' ') for spker in raw_data.SPEAKER] for s in tkns if len(s) <= 3 and s not in ['위원장', '위원', '의원', '진술인', '참고인']])
manual_rms = np.append(manual_rms, ["계속", '료', '박정', '말씀', '생각', '얘기', '질의', '발언', '국민'])

raw_data.token = [[[tkn for tkn in doc if tkn not in manual_rms] for doc in corp] for corp in tqdm(raw_data.token)]
raw_data.token = [[doc for doc in corp if doc != []] for corp in tqdm(raw_data.token)]

path = '200427_0005_20대_보건복지부_발언정제'
'./result/'
flda = gs.models.LdaMulticore.load(f'./result/{path}/ldafit.pickle')
fid2word = gs.models.LdaMulticore.load(f'./result/{path}/ldafit.pickle.id2word')
topiced_terms = set([flda.id2word[term] for t in range(flda.num_topics) for term,p in flda.get_topic_terms(t)])

raw_data.token = [[[tkn for tkn in doc if tkn in topiced_terms] for doc in corp] for corp in tqdm(raw_data.token)]
raw_data = raw_data.loc[[i for i, d in raw_data.token.items() if d != []]]

In [66]:
raw_data.tail(3)

,CONFER_NUM,MEETING_NAME,SEQ_NO,AGENDA,SPEAKER,PARTY,CONTENT,token
5294,48725,제364회(2018.09.01-2018.12.09),11,1. 2019년도 예산안(계속)가. 보건복지부 소관나. 식품의약품안전처 소관2. 2...,남인순 위원,더불어민주당,다함께돌봄사업을 지역아동센터도 오랫동안 어렵게 현장에서 왔던 그룹들이기 부분과 균형...,"[[사업, 아동], [], []]"
5295,48725,제364회(2018.09.01-2018.12.09),13,1. 2019년도 예산안(계속)가. 보건복지부 소관나. 식품의약품안전처 소관2. 2...,남인순 위원,더불어민주당,지침을 내려보내 주십시오.,"[[], []]"
5296,48725,제364회(2018.09.01-2018.12.09),15,1. 2019년도 예산안(계속)가. 보건복지부 소관나. 식품의약품안전처 소관2. 2...,남인순 위원,더불어민주당,현장에서 혼선이 있습니다.,"[[], []]"


#### Ngram model

In [67]:
ngram = 1
raw_data.token = [make_ngram(doc, ngram, mincnt = 1, thrsh = 2) for doc in tqdm(raw_data.token)]

100%|█████████████████████████████████████████████████████████████████████████| 9621/9621 [00:00<00:00, 1203357.75it/s]


In [68]:
보건복지부 = {}
for i, (전후, 회차) in enumerate([('전체', [343, 364]), ('정권교체전', [343, 351]), ('정권교체후', [352, 364])]):
    gdata = raw_data.loc[[idx
                          for n in range(회차[0],회차[1]+1)
                          for idx, m in zip(raw_data.MEETING_NAME.index, raw_data.MEETING_NAME) # filter by meetings
                          if m.find(str(n)) > -1]]
    doC =  {'{}'.format(p):my_list(gdata.loc[[idx for idx in gdata.index
                                              if gdata.loc[idx].PARTY == p]].token).sum() # filter by parties
            for p in np.unique(gdata.PARTY)}
    doC.update({'정당통합':[d for _, C in doC.items() for d in C]})
    보건복지부[전후] = doC

for k1, i1 in 보건복지부.items():
    for k2, i2 in i1.items():
        보건복지부[k1][k2] = [d for d in i2 if d != []]

#### 21대 총선

In [ ]:
ngram = 1
project = '21대_총선'
#     1. Data Preparation
doC = pd.read_pickle('./data/election_21st.pickle')

rms = ['정책', '개혁', '추진'] # cliche
총선 = {p:None for p in doC}
for p in 총선:
    총선[p] = [[w for w in d if w not in rms] for d in doC[p]]
    총선[p] = [d for d in 총선[p] if d != []]
    총선[p] = make_ngram(총선[p], ngram, mincnt = 1, thrsh = 2)
총선.update({'정당통합':my_list([d for _, d in 총선.items()]).sum()})

#### 문재인 19대 대선

In [ ]:
ngram = 1
project = '19대_대선_문재인'
#     1. Data Preparation
doc = pd.read_pickle('./data/MJI19thPE.pickle')

rms = [''] # cliche
문재인 = {'문재인':None}
문재인['문재인'] = [[w for w in d if w not in rms] for d in doc]
문재인['문재인'] = [d for d in 문재인['문재인'] if d != []]
문재인['문재인'] = make_ngram(문재인['문재인'], ngram, mincnt = 1, thrsh = 2)

* 보건복지위 샘플

In [69]:
pprint([f'{k1}/{k2}: {len(i2)} lines, {i2[2:4]}' for k1, i1 in 보건복지부.items() for k2, i2 in i1.items()])

["전체/국민의당: 2434 lines, [['보건', '복지', '위원장', '위원장'], ['위원장']]",
 "전체/더불어민주당: 10716 lines, [['보건', '복지', '위원회', '위원'], ['보건', '복지', '위원', "
 "'위원장']]",
 "전체/무소속: 1604 lines, [['복지', '문제'], ['위원']]",
 "전체/자유한국당: 6014 lines, [['위원회', '보건', '복지', '위원회', '위원회'], ['위원장', '간사', "
 "'간사']]",
 "전체/정당통합: 20768 lines, [['보건', '복지', '위원장', '위원장'], ['위원장']]",
 "정권교체전/국민의당: 1151 lines, [['보건', '복지', '위원장', '위원장'], ['위원장']]",
 "정권교체전/더불어민주당: 5683 lines, [['보건', '복지', '위원회', '위원'], ['보건', '복지', '위원', "
 "'위원장']]",
 "정권교체전/무소속: 815 lines, [['복지', '문제'], ['위원']]",
 "정권교체전/자유한국당: 1988 lines, [['위원회', '보건', '복지', '위원회', '위원회'], ['위원장', '간사', "
 "'간사']]",
 "정권교체전/정당통합: 9637 lines, [['보건', '복지', '위원장', '위원장'], ['위원장']]",
 "정권교체후/국민의당: 1283 lines, [['의결', '심사'], ['노인', '건강', '지원']]",
 "정권교체후/더불어민주당: 5033 lines, [['조사'], ['조사']]",
 "정권교체후/무소속: 789 lines, [['위원', '위원'], ['진행', '보건', '복지', '위원회', '사람', '위원']]",
 "정권교체후/자유한국당: 4026 lines, [['의결', '장관', '후보', '협의'], ['위원장', '장관', '후보']]",
 "정권교체후/정당통합: 11131 lines

* 총선 샘플

In [8]:
pprint([f'{k}: {len(i)} lines, {i[:2]}' for k,i in 총선.items()][:5])

["더불어민주당: 452 lines, [['벤처', '강국', '실현'], ['중소', '벤처']]",
 "미래통합당: 462 lines, [['국민', '희망', '공약', '개발', '단'], ['핵심', '공약']]",
 "민생당: 293 lines, [['코로나', '피해', '중소', '상공', '영업자'], ['民生', '지원', '극복', "
 "'수당', '지급']]",
 "친박신당: 306 lines, [['자유민주주의', '수호', '개헌', '반대', '정치'], ['박근혜', '대통령', "
 "'잘못', '탄핵', '자유민주주의', '기본', '질서', '위협']]",
 "국민의당: 316 lines, [['정치', '국회법', '정당법', '일', '정치', '실현'], ['정치', '권력', "
 "'사유', '민주', '통제', '강화']]"]


* 문재인 샘플

In [9]:
pprint([f'{k}: {len(i)} lines, {i[:2]}' for k,i in 문재인.items()])

["문재인: 495 lines, [['일자리', '대한민국', '공약', '분야'], ['노동']]"]


## Co-occurence TFIDF 

In [70]:
import copy
def maxdiagindx(dataframe, m, dsc = True):
    dranks = sorted([idx for idx in dataframe.index], key=lambda x: dataframe.loc[x][x], reverse=dsc)
    return(dranks[m[0]:m[1]])

In [71]:
Data = {f'보건복지{p}':C for p, C in 보건복지부.items()}
# Data = {'총선':총선}
# Data = {'문재인':문재인}

In [72]:
# TFIDF = copy.deepcopy(Data)
coocs = copy.deepcopy(Data)
keys = []
for prj, datum in Data.items(): # 보복전, dict of dicts of lists
    for party, tokens in tqdm(datum.items()): # 국민의당, list of lists
        norm_corp = [my_list(t).sumtkns() for t in tokens]
        cv = CountVectorizer()
        X = cv.fit_transform(norm_corp)
        if prj.find('보건복지') > -1:
            start = 0
            if party == '정당통합':
                m = [start, start + 500]
            else:
                m = [start, start + 500]
        else:
            if party == '정당통합':
                m = [1, 201]
            else:
                m = [1, 101]
                
        Xc = (X.T * X) # This is the matrix manipulation step
        names = cv.get_feature_names() # This are the entity names (i.e. keywords)
        cooc = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
        dranks = maxdiagindx(cooc, m)
        cooc.values[(np.arange(cooc.shape[0]), np.arange(cooc.shape[0]))] = 0
        cooc = cooc.loc[dranks][dranks]
#         print(dranks)
        coocs[prj][party] = cooc

#         2.3. Fit TFIDF
#         tv = TfidfVectorizer(use_idf=True, min_df = .0, max_df= 1.)
#         tv_matrix = tv.fit_transform(norm_corp).toarray()
#         vocab = tv.get_feature_names()
#         TFIDF[prj][party] = pd.DataFrame(tv_matrix, columns=vocab)

#         2.4. Cosine Similarity

#         X = TFIPF
#         X = tfidf.apply(lambda x: x/norm(x,1), axis=0)
#         CS = X.T.dot(X)
#         np.fill_diagonal(CS.values, 0) # Set diag. zeros

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 19.85it/s]


In [73]:
[c.to_csv('./Gephi/csvs/' + prj + party + '.csv', encoding = 'euc-kr')
 for prj, datum in coocs.items()
 for party, c in datum.items()]
print('Csvs saved!')

Csvs saved!


## 보건복지부 Dynamic Network

In [ ]:
import matplotlib.font_manager as fm
from matplotlib import rc

font_name = fm.FontProperties(fname="/Users/jangjaeho/Library/fonts/BMDOHYEON_ttf.ttf").get_name()
rc('font', family=font_name)

# sorted([f.name for f in fm.fontManager.ttflist])

In [ ]:
# for i, k in enumerate(keys[:1]):
#     print(k)
    
#     plt.figure(figsize=(30, 30))
#     Gdata = coocs[keys[i][0]][keys[i][1]]
# #     print(Gdata.iloc[:5,:5])
#     G = nx.Graph(Gdata)
#     options = {
#         'node_color': 'green',
#         'node_size': 5000,
#         'width': .1,
#         'alpha': 0.4
#     }
#     # G.nodes.data()
    
# #     Generating gefx
#     # linefeed = chr(10)
#     # s = linefeed.join(nx.generate_gexf(G))
    
# #     1. Ordinary spring layout
    
#     layouts = [nx.spring_layout, nx.bipartite_layout, nx.kamada_kawai_layout,
#                nx.shell_layout, nx.spectral_layout, nx.spiral_layout]
#     pos = layouts[0](G)
# #     pos = graphviz_layout(G, 'twopi')
#     nx.draw(G, pos, **options)
# #     nx.draw_networkx()
# #     nx.draw_spectral(G, **options)
# #     nx.draw_networkx_labels(G, pos, font_family = 'AppleGothic', font_size=30) # For MAC
#     nx.draw_networkx_labels(G, pos, font_family = 'HCR Batang', font_size=25)
    
# #     2. Hierarchical Cluster of graph G

# #     # Extract largest connected component into graph H
# #     H = G.subgraph(next(nx.connected_components(G)))
# #     # Makes life easier to have consecutively labeled integer nodes
# #     H = nx.convert_node_labels_to_integers(H)
# #     # Create parititions with hierarchical clustering
# #     partitions = create_hc(H)
# #     # Build blockmodel graph
# #     BM = nx.quotient_graph(H, partitions, relabel=True)

# #     # Draw original graph
# #     pos = nx.spring_layout(H, iterations=100)
# #     plt.subplot(211)
# #     nx.draw(H, pos, **options)
# #     nx.draw_networkx_labels(H, pos, font_family='HCR Batang', font_size=30)

# #     # Draw block model with weighted edges and nodes sized by number of internal nodes
# #     node_size = [BM.nodes[x]['nnodes'] * 10 for x in BM.nodes()]
# #     edge_width = [(2 * d['weight']) for (u, v, d) in BM.edges(data=True)]
# #     # Set positions to mean of positions of internal nodes from original graph
# #     posBM = {}
# #     for n in BM:
# #         xy = np.array([pos[u] for u in BM.nodes[n]['graph']])
# #         posBM[n] = xy.mean(axis=0)
# #     plt.subplot(212)
# #     nx.draw(BM, posBM, **options)
# #     nx.draw_networkx_labels(BM, posBM, font_family='HCR Batang', font_size=30)
    
# #     A. Statistics
# #     print("Betweenness")
# #     b = nx.betweenness_centrality(G)
# #     for v in G.nodes():
# #         print((v, b[v]))

# #     print("Degree centrality")
# #     d = nx.degree_centrality(G)
# #     for v in G.nodes():
# #         print("%0.2d %5.3f" % (v, d[v]))

# #     print("Closeness centrality")
# #     c = nx.closeness_centrality(G)
# #     for v in G.nodes():
# #         print("%0.2d %5.3f" % (v, c[v]))

#     plt.axis('equal')
#     plt.savefig('./Gephi/figures/' + k[0] + k[1] + '.png', )
# #     plt.show()